<a href="https://colab.research.google.com/github/ephantusGachomba/To_Vaccinate_or_not_to_Vaccinate_NLP/blob/main/To_Vaccinate_or_not_to_Vaccinate_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install simpletransformers

In [2]:
#Importing the basic libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import *
from sklearn.model_selection import *

from tqdm import tqdm # Python library for adding progress bar
import warnings
warnings.simplefilter('ignore')
import gc #automatically releases memory when an object is no longer used. 
from scipy.special import softmax

from simpletransformers.classification.classification_model import ClassificationModel

#Evaluation of the model
from sklearn.metrics import mean_squared_error as mse

In [3]:
train_df = pd.read_csv('Train.csv')
test_df = pd.read_csv('Test.csv')
sample_sub = pd.read_csv('SampleSubmission.csv')
ID_COL, TARGET_COL = sample_sub.columns.tolist()

In [4]:
train_df.head()

,tweet_id,safe_text,label,agreement
0,CL1KWCMY,Me &amp; The Big Homie meanboy3000 #MEANBOY #M...,0.0,1.0
1,E3303EME,I'm 100% thinking of devoting my career to pro...,1.0,1.0
2,M4IVFSMS,"#whatcausesautism VACCINES, DO NOT VACCINATE Y...",-1.0,1.0
3,1DR6ROZ4,I mean if they immunize my kid with something ...,-1.0,1.0
4,J77ENIIE,Thanks to <user> Catch me performing at La Nui...,0.0,1.0


In [5]:
test_df.head()

,tweet_id,safe_text
0,00BHHHP1,<user> <user> ... &amp; 4 a vaccine given 2 he...
1,00UNMD0E,Students starting school without whooping coug...
2,01AXPTJF,"I'm kinda over every ep of <user> being ""rippe..."
3,01HOEQJW,How many innocent children die for lack of vac...
4,01JUKMAO,"CDC eyeing bird flu vaccine for humans, though..."


In [6]:
sample_sub.head()

,tweet_id,label
0,00BHHHP1,0
1,00UNMD0E,0
2,01AXPTJF,0
3,01HOEQJW,0
4,01JUKMAO,0


In [7]:
train_df['safe_text'].apply(lambda x: len(x)).describe()

count    10001.000000
mean        99.902810
std         29.893888
min          1.000000
25%         79.000000
50%        107.000000
75%        122.000000
max        153.000000
Name: safe_text, dtype: float64

In [8]:
train_df['label'].value_counts()

 0.000000    4908
 1.000000    4053
-1.000000    1038
 0.666667       1
Name: label, dtype: int64

In [9]:
#Replacing the 0.666667 label with -1
train_df['label'][~train_df['label'].isin([0, -1, 1])] = -1

In [10]:
#Check for null values
train_df.isnull().sum()

tweet_id     0
safe_text    0
label        0
agreement    2
dtype: int64

In [11]:
#dropping the null values
train_df.dropna(inplace=True)

In [12]:
#Check for null values
train_df.isnull().sum()

tweet_id     0
safe_text    0
label        0
agreement    0
dtype: int64

In [13]:
#Cleaning test data

In [14]:
test_df.head()

,tweet_id,safe_text
0,00BHHHP1,<user> <user> ... &amp; 4 a vaccine given 2 he...
1,00UNMD0E,Students starting school without whooping coug...
2,01AXPTJF,"I'm kinda over every ep of <user> being ""rippe..."
3,01HOEQJW,How many innocent children die for lack of vac...
4,01JUKMAO,"CDC eyeing bird flu vaccine for humans, though..."


In [15]:
test_df.isnull().sum()

tweet_id     0
safe_text    1
dtype: int64

In [16]:
test_df['safe_text'] = test_df['safe_text'].fillna('xxxxxx')

In [17]:
test_df.isnull().sum()

tweet_id     0
safe_text    0
dtype: int64

## Sampling Each of the tweets

In [18]:
train_df.head()

,tweet_id,safe_text,label,agreement
0,CL1KWCMY,Me &amp; The Big Homie meanboy3000 #MEANBOY #M...,0.0,1.0
1,E3303EME,I'm 100% thinking of devoting my career to pro...,1.0,1.0
2,M4IVFSMS,"#whatcausesautism VACCINES, DO NOT VACCINATE Y...",-1.0,1.0
3,1DR6ROZ4,I mean if they immunize my kid with something ...,-1.0,1.0
4,J77ENIIE,Thanks to <user> Catch me performing at La Nui...,0.0,1.0


#### 1. Positive

In [19]:
train_df[train_df['label']==1]['safe_text'].values[:5]

array(["I'm 100% thinking of devoting my career to proving autism isn't caused by vaccines due to the IDIOTIC posts I've seen about World Autism Day",
       '<user> a nearly 67 year old study when mental health studies and vaccines were relatively in their infancies that has been refuted?',
       'Study of more than 95,000 kids finds no link between MMR vaccine and autism <url>',
       'psa: VACCINATE YOUR FUCKING KIDS',
       'Coughing extra on the shuttle and everyone thinks I have the measles. 😂 #VaccinateYourKids'],
      dtype=object)

#### 2.Neutral

In [20]:
train_df[train_df['label']==0]['safe_text'].values[:5]

array(['Me &amp; The Big Homie meanboy3000 #MEANBOY #MB #MBS #MMR #STEGMANLIFE @ Stegman St. <url>',
       'Thanks to <user> Catch me performing at La Nuit NYC 1134 1st ave. Show starts at 6! #jennifair #mmr… <url>',
       "<user> @ this point I have 2 text, butw/Bon Jovi cover playin @ Alibi's hope U can come out 2 MMR BBQ<user> will b there!",
       'My prediction, vaccine exemption in Arizona will end soon. To much money is being lost by big pharma.',
       '1$Mug Noche <user> #mmr #mixmasterrod #dcdj #mmr   @ Mad Hatter <url>'],
      dtype=object)

#### 3. Negative

In [21]:
train_df[train_df['label']==-1]['safe_text'].values[:5]

array(['#whatcausesautism VACCINES, DO NOT VACCINATE YOUR CHILD',
       "I mean if they immunize my kid with something that won't secretly kill him years down the line then I'm all for it, but I don't trust that",
       '<user> #CDC lied and hid data that black boys have an 340% uncreased risk of developing autism after MMR #CDCwhistleblower help!?!?',
       '<user> vaccines causing autism',
       '<user> <user> Other than that, his defense is not against vaccines being harmful - but the American life being unhealthy.'],
      dtype=object)

Lets go to the modelling part. Simple Transformers is extremely simple to use, and switching architectures requires only the change in argument name.

In [22]:
def get_model(model_type, model_name, n_epochs = 2, train_batch_size = 112, eval_batch_size = 144, seq_len = 134, lr = 2e-5, use_cuda = False):
  model = ClassificationModel(model_type, model_name,num_labels=1, args={'train_batch_size':train_batch_size,
                                                                         "eval_batch_size": eval_batch_size,
                                                                         'reprocess_input_data': True,
                                                                         'overwrite_output_dir': True,
                                                                         'fp16': False,
                                                                         'do_lower_case': False,
                                                                         'num_train_epochs': n_epochs,
                                                                         'max_seq_length': seq_len,
                                                                         'regression': True,
                                                                         'manual_seed': 2,
                                                                         "learning_rate":lr,
                                                                         "save_eval_checkpoints": False,
                                                                         "save_model_every_epoch": False,},
                              use_cuda = False)
  return model

In [23]:
tmp = pd.DataFrame()
tmp['text'] = train_df['safe_text']
tmp['labels'] = train_df['label']
tmp_test = test_df[['safe_text']].rename({'safe_text': 'text'}, axis=1)
tmp_test['labels'] = 0
tmp_trn, tmp_val = train_test_split(tmp, test_size=0.15, random_state=2)

## Model A: Roberta Base 3 Epochs

In [ ]:
model = get_model('roberta', 'roberta-base', n_epochs=3)
model.train_model(tmp_trn)
preds_val = model.eval_model(tmp_val)[1]
preds_val = np.clip(preds_val, -1, 1)
print(f"RMSE: {mse(tmp_val['labels'], preds_val)**0.5}")
test_preds = model.eval_model(tmp_test)[1]
test_preds = np.clip(test_preds, -1, 1)
pv_1 = preds_val
pt_1 = test_preds

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.weight', 'lm_head.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classi

  0%|          | 0/8499 [00:00<?, ?it/s]

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/76 [00:00<?, ?it/s]

## Model B: Roberta Large 1 Epoch

In [ ]:
model = get_model('roberta', 'roberta-large', n_epochs=1, train_batch_size=16, eval_batch_size=16)
model.train_model(tmp_trn)
preds_val = model.eval_model(tmp_val)[1]
preds_val = np.clip(preds_val, -1, 1)
print(f"RMSE: {mse(tmp_val['labels'], preds_val)**0.5}")
test_preds = model.eval_model(tmp_test)[1]
test_preds = np.clip(test_preds, -1, 1)
pv_2 = preds_val
pt_2 = test_preds

## Model C: Roberta Large 2 Epochs

In [ ]:
model = get_model('roberta', 'roberta-large', n_epochs=2, train_batch_size=16, eval_batch_size=16, lr = 2e-5)
model.train_model(tmp_trn)
preds_val = model.eval_model(tmp_val)[1]
preds_val = np.clip(preds_val, -1, 1)
print(f"RMSE: {mse(tmp_val['labels'], preds_val)**0.5}")
test_preds = model.eval_model(tmp_test)[1]
test_preds = np.clip(test_preds, -1, 1)
pv_3 = preds_val
pt_3 = test_preds

## Model D: Roberta Large 3 Epochs

In [1]:
model = get_model('roberta', 'roberta-large', n_epochs=3, train_batch_size=16, eval_batch_size=16, lr = 1e-5)
model.train_model(tmp_trn)
preds_val = model.eval_model(tmp_val)[1]
preds_val = np.clip(preds_val, -1, 1)
print(f"RMSE: {mse(tmp_val['labels'], preds_val)**0.5}")
test_preds = model.eval_model(tmp_test)[1]
test_preds = np.clip(test_preds, -1, 1)
pv_4 = preds_val
pt_4 = test_preds

NameError: ignored

## Blending of Different Models

In [ ]:
pv = ((pv_1 * 0.3 + pv_2 * 0.7) * 0.3 + pv_3*0.7)*0.65 + pv_4*0.35
print(f"RMSE: {mse(tmp_val['labels'], pv)**0.5}")

In [ ]:
tp = ((pt_1 * 0.3 + pt_2 * 0.7) * 0.3 + pt_3*0.7)*0.65 + pt_4*0.35

In [ ]:
pd.Series(tp).describe()

In [ ]:
final_test_preds = tp
preds_df_final = test[[ID_COL]]
preds_df_final[TARGET_COL] = final_test_preds
SUB_FILE_NAME = 'roberta_ensemble.csv'
preds_df_final.to_csv(SUB_FILE_NAME, index=False)

In [ ]:
preds_df_final.head()

In [ ]:
preds_df_final.tail()

In [ ]:
preds_df_final.shape

In [ ]:
preds_df_final['label'].describe()

In [ ]:
preds_df_final.isnull().sum()

In [ ]:
from google.colab import files
files.download(SUB_FILE_NAME)